# Gradient Descent

Gradient Descent is an algorithm that minimizes some objective $J(\theta)$ where $\theta$ is $\in \mathbb{R}^d$ by iteratively updating the vector $\theta$ by moving in the direction of steepest descent multiplied by some step size or learning rate.

In essence, 

$\>\>\>\>$1: Pick an initial guess $\theta_0$    
$\>\>\>\>$2: $\>\>$while $J(\theta_{k+1}) - J(\theta_{k}) \leq precision$:    
$\>\>\>\>$3: $\>\>\>\>\>\>\>\>$ $\theta_{k+1}$ = $\theta_{k} -\nabla J_\theta(\theta_k)$ * $\alpha_k$.

This is relevant to Machine Learning because it offers a numerical solution to estimates of parameters that do not have a closed form solution.

## Estimating Multinomial Logistic Regression Parameters with Gradient Descent 

#### Cost Function
However, one needs to determine the objective $J(\theta)$ before executing this algorithm. In the case of linear regression, the sum of squared residuals could be used. In this case, one can use the idea of MLE to minimize the log liklihood.

In the case where $K = 2$, the likelihood function was $\prod_{i=1}^{n} p(\textbf x_i)^{y_i} (1-p(\textbf x_i))^{1-y_i}$ 

In the case where $K > 2$,

For $k \in K$ where $K$ is the set of labeled classes and $p$ is the number of features      
Let $p_k(\textbf x)$ be the posterior probability $P(Y = k | X = \bf x)$ and the model be defined as 

$p_k(\textbf x) = \frac{e^{\beta_{0}^k + \textbf w_k\cdot \textbf x}}{\sum_{k=1}^{K} e^{\beta_{0}^k + \textbf w_k\cdot \textbf x}}$ for $k=1...K$

One can see how each class $1 \leq k \leq K$ has its own set of parameters $w_k = (\beta_1,...,\beta_p)$

We want the optimal set of parameters(weights): argmax $L(\beta$ $|$ $ \textbf x)$

$$\prod_{k=1}^{K}P(Y = k | X = \textbf x_i)^{k_i} = \prod_{k=1}^{K} p_k(\textbf x_i)^{k_i}$$

Minimizing the negative log-liklihood is the same as maximizing the log-liklihood and can be taken as an average over an entire sample

$$J(\textbf w) = \frac{-1}{n}log\prod_{i=1}^{N}\prod_{k=1}^{K} p_k(\textbf x_i)^{k_i} = \frac{-1}{n}\sum_{i=1}^{N}\sum_{k=1}^{K} {k_i} * log(\hat{y}_{ki})$$

where $k_i$ is 1 if $\textbf x_i$ is labeled as class k

Finally, we introduce the L2 norm to obtain a regularized objective function.

$$J(\textbf w) = \frac{-1}{n}log\prod_{i=1}^{N}\prod_{k=1}^{K} p_k(\textbf x_i)^{k_i} = \frac{-1}{n}\sum_{i=1}^{N}\sum_{k=1}^{K} {k_i} * log(\hat{y}_{ki}) + \alpha R(\textbf w)$$

where $R(\textbf w) = ||\beta||^2_2 = \sum^{k}_{i=1}\beta_i^2$ and $p_k(\textbf x_i)^{k_i} = \hat{y}_{ki}$
#### Gradient of Cost Function

TODO

In [1]:
import numpy as np

In [125]:
class BatchGradientDescent:
    def __init__(self):
        self.rate = .1
        self.precision = .0001
        self.reg = .1
    
    # Pre: X - design matrix
    #      y - labeled classes
    #      w - pxk parameter matrix of current model
    # Post: a vector of K errors,one for each label
    def determineCost(self, X, y, w,mini_batch = False):
        yhat = self.softmax(np.dot(X,w))
        # sum K sets of squared beta_i
        R_w = self.reg * np.sum(w * w, axis = 0)
        # * operator will ensure a valid summation over all samples because the labels have been one-hot encoded
        J = -np.sum((y * np.log(yhat)),axis = 0)+R_w
        return J/X.shape[0]
    
    # Pre: X - design matrix
    #      y - labeled classes
    #      w - pxk parameter matrix of current model
    # Post: the gradient of J evaluated at w
    def gradient(self, X, y, w,mini_batch = False):
        if(mini_batch == True):
            X = X[np.random.choice(X.shape[0], 2, replace=False), :]
        yhat = self.softmax(np.dot(X,w))
        gradient = np.dot(X.transpose(),(yhat-y))+self.reg * w
        return gradient/X.shape[0]
    
    # Pre: z - x dot w of dimension (n*p)multiply(p*k=10)=>n*10
    # Post: Softmax score for each observation
    def softmax(self,z):
        return np.exp(z-np.max(z)) / np.sum(np.exp(z-np.max(z)), axis=1, keepdims=True)
    
    def error(self,X, y, w):
        yhat = self.softmax(np.dot(X,w))
        return np.sum(np.argmax(yhat,axis=1)!=np.argmax(y,axis=1))/float(X.shape[0]) 
    
    # Pre: X - design matrix
    #      y - labeled classes
    #      w - pxk parameter matrix of current model
    # Post: return optimal set of weights w, J(theta) over training set,accuracy over training set
    def gradientDescent(self,X,y,w):
        rate = self.rate
        iteration = 0
        J = []
        accuracy = []
        J.append(np.sum(np.array([0]*10)))
        J.append(np.sum(self.determineCost(X,y,w)))
        while(np.abs(J[-1]-J[-2]) > self.precision):
            if((len(J)-1)%10==0):
                accuracy.append(1-self.error(X,y,w))
                print(accuracy)
            w = w - self.rate * self.gradient(X,y,w)
            J.append(np.sum(self.determineCost(X,y,w)))
            iteration = iteration + 1
            rate = rate * 1/(1 + .001 * iteration)
        return w,J,accuracy

In [95]:
from sklearn.preprocessing import OneHotEncoder
class LogisticRegression:
    
    # Pre - data matrix X
    # Post - data matrix X with bias term concatenated as first column 
    def addBias(self,X):
        X = np.concatenate((np.ones((X.shape[0],1)),X),axis=1)
        return X
    
    def normalize(self,X):                                                     
        X = (X - np.min(X))/float(np.max(X) - np.min(X))
        return X
    
    def oneHotEncode(self,y):
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(y)
        return np.array(enc.transform(y).toarray())
    
    def train(self,trainX,trainy):
        trainX = self.normalize(trainX)
        trainX = self.addBias(trainX)
        trainy = self.oneHotEncode(trainy)
        
        p,k = trainX.shape[1],trainy.shape[1]
        w = np.random.rand(p,k)
        print("Training Started")
        start_time = time()
        GD = BatchGradientDescent()
        w,J,accuracy = GD.gradientDescent(trainX,trainy,w)
        end_time = time()
        print("Training Finished, Took: %ss"%((end_time-start_time)))
        return w,J,accuracy

In [119]:
# loading the mnist dataset
from keras.datasets import mnist
from matplotlib import pyplot
# load dataset
(trainX, trainy), (testX, testy) = mnist.load_data()

trainX = trainX.reshape((trainX.shape[0],trainX.shape[1]*trainX.shape[1]))
trainy = trainy.reshape((trainX.shape[0],1))

testX = testX.reshape((testX.shape[0],testX.shape[1]*testX.shape[1]))
testy = testy.reshape((testX.shape[0],1))

# summarize loaded and reshaped dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

Train: X=(60000, 784), y=(60000, 1)
Test: X=(10000, 784), y=(10000, 1)


In [ ]:
logreg = LogisticRegression()
w,J,accuracy = logreg.train(trainX,trainy)

Training
[0.21989999999999998]
[0.21989999999999998, 0.35048333333333337]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666, 0.83795, 0.8396166666666667, 0.8414833333333334, 0.8430333333333333, 0.84465, 0.8463666666666667, 0.8476166666666667, 0.8488666666666667, 0.8502166666666666, 0.8512833333333334, 0.8524, 0.8534333333333333, 0.8545]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.75021

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666, 0.83795, 0.8396166666666667, 0.8414833333333334, 0.8430333333333333, 0.84465, 0.8463666666666667, 0.8476166666666667, 0.8488666666666667, 0.8502166666666666, 0.8512833333333334, 0.8524, 0.8534333333333333, 0.8545, 0.85555, 0.85655, 0.8575, 0.8584333333333334, 0.8593666666666666, 0.86025, 0.8610833333333333, 0.8619333333333333, 0.8627166666666667, 0.8634333333333333]
[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666, 0.83795, 0.8396166666666667, 0.8414833333333334, 0.8430333333333333, 0.84465, 0.8463666666666667, 0.8476166666666667, 0.8488666666666667, 0.8502166666666666, 0.8512833333333334, 0.8524, 0.8534333333333333, 0.8545, 0.85555, 0.85655, 0.8575, 0.8584333333333334, 0.8593666666666666, 0.86025, 0.8610833333333333, 0.8619333333333333, 0.8627166666666667, 0.8634333333333333, 0.864, 0.8647833333333333, 0.8655833333333334, 0.866166666666666

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666, 0.83795, 0.8396166666666667, 0.8414833333333334, 0.8430333333333333, 0.84465, 0.8463666666666667, 0.8476166666666667, 0.8488666666666667, 0.8502166666666666, 0.8512833333333334, 0.8524, 0.8534333333333333, 0.8545, 0.85555, 0.85655, 0.8575, 0.8584333333333334, 0.8593666666666666, 0.86025, 0.8610833333333333, 0.8619333333333333, 0.8627166666666667, 0.8634333333333333, 0.864, 0.8647833333333333, 0.8655833333333334, 0.866166666666666

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666, 0.83795, 0.8396166666666667, 0.8414833333333334, 0.8430333333333333, 0.84465, 0.8463666666666667, 0.8476166666666667, 0.8488666666666667, 0.8502166666666666, 0.8512833333333334, 0.8524, 0.8534333333333333, 0.8545, 0.85555, 0.85655, 0.8575, 0.8584333333333334, 0.8593666666666666, 0.86025, 0.8610833333333333, 0.8619333333333333, 0.8627166666666667, 0.8634333333333333, 0.864, 0.8647833333333333, 0.8655833333333334, 0.866166666666666

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666, 0.83795, 0.8396166666666667, 0.8414833333333334, 0.8430333333333333, 0.84465, 0.8463666666666667, 0.8476166666666667, 0.8488666666666667, 0.8502166666666666, 0.8512833333333334, 0.8524, 0.8534333333333333, 0.8545, 0.85555, 0.85655, 0.8575, 0.8584333333333334, 0.8593666666666666, 0.86025, 0.8610833333333333, 0.8619333333333333, 0.8627166666666667, 0.8634333333333333, 0.864, 0.8647833333333333, 0.8655833333333334, 0.866166666666666

[0.21989999999999998, 0.35048333333333337, 0.4588833333333333, 0.5389833333333334, 0.5930500000000001, 0.6332333333333333, 0.6645166666666666, 0.6889833333333333, 0.7080166666666667, 0.72435, 0.7383333333333333, 0.7502166666666666, 0.7604166666666666, 0.7693666666666666, 0.7771833333333333, 0.78335, 0.7891, 0.7943166666666667, 0.7996333333333333, 0.8039333333333334, 0.8080833333333333, 0.8116166666666667, 0.8150333333333333, 0.81795, 0.8213666666666667, 0.8246333333333333, 0.8272166666666667, 0.82955, 0.8319333333333333, 0.8340666666666667, 0.8358666666666666, 0.83795, 0.8396166666666667, 0.8414833333333334, 0.8430333333333333, 0.84465, 0.8463666666666667, 0.8476166666666667, 0.8488666666666667, 0.8502166666666666, 0.8512833333333334, 0.8524, 0.8534333333333333, 0.8545, 0.85555, 0.85655, 0.8575, 0.8584333333333334, 0.8593666666666666, 0.86025, 0.8610833333333333, 0.8619333333333333, 0.8627166666666667, 0.8634333333333333, 0.864, 0.8647833333333333, 0.8655833333333334, 0.866166666666666